In [1]:
import cv2, os, glob
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Conv2D, Input, Activation
from keras.models import Model
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from skimage.transform import pyramid_expand
from Subpixel import Subpixel
from DataGenerator import DataGenerator
from Modimg import ModImg

Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [2]:
base_path = 'D:\\Data'

x_train_list = sorted(glob.glob(os.path.join(base_path, 'x_train', '*.npy')))
x_val_list = sorted(glob.glob(os.path.join(base_path, 'x_val', '*.npy')))

print(len(x_train_list), len(x_val_list))
print(x_train_list[0])

4928 1133
D:\Data\x_train\0.npy


In [3]:
train_gen = DataGenerator(list_IDs=x_train_list, labels=None, batch_size=4, dim=(256,256), n_channels=3, n_classes=None, shuffle=True)

val_gen = DataGenerator(list_IDs=x_val_list, labels=None, batch_size=4, dim=(256,256), n_channels=3, n_classes=None, shuffle=False)

In [4]:
upscale_factor = 2

inputs = Input(shape=(256, 256, 3))

net = Conv2D(filters=64, kernel_size=5, strides=1, padding='same', activation='relu')(inputs)
net = Conv2D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu')(net)
net = Conv2D(filters=32, kernel_size=3, strides=1, padding='same', activation='relu')(net)
net = Conv2D(filters=upscale_factor**2, kernel_size=3, strides=1, padding='same', activation='relu')(net)
net = Subpixel(filters=3, kernel_size=3, r=upscale_factor, padding='same')(net)
outputs = Activation('relu')(net)

model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam', loss='mse')

model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 256, 256, 64)      4864      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 256, 256, 64)      36928     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 256, 256, 32)      18464     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 256, 256, 4)       1156      
_________________________________________________________________
subpixel_1 (Subpixel)        (None, 512, 512, 3)       444       
_________________________________________________________________
activation_1 (Activation)    (None, 512, 512, 3)       0   

In [6]:
model = load_model('models/model_256.h5', custom_objects={'Subpixel':Subpixel})

In [ ]:
history = model.fit_generator(train_gen, validation_data=val_gen, epochs=20, verbose=1, callbacks=[
    ModelCheckpoint('models/model_256.h5', monitor='val_loss', verbose=1, save_best_only=True)
])

Epoch 1/20
1232/1232 [==============================] - 3063s 2s/step - loss: 0.0022 - val_loss: 0.0018

Epoch 00001: val_loss improved from inf to 0.00176, saving model to models/model_256.h5
Epoch 2/20
1232/1232 [==============================] - 3217s 3s/step - loss: 0.0022 - val_loss: 0.0017

Epoch 00002: val_loss improved from 0.00176 to 0.00169, saving model to models/model_256.h5
Epoch 3/20
1232/1232 [==============================] - 3225s 3s/step - loss: 0.0021 - val_loss: 0.0028

Epoch 00003: val_loss did not improve from 0.00169
Epoch 4/20
1232/1232 [==============================] - 3289s 3s/step - loss: 0.0020 - val_loss: 0.0015

Epoch 00004: val_loss improved from 0.00169 to 0.00155, saving model to models/model_256.h5
Epoch 5/20
1232/1232 [==============================] - 3335s 3s/step - loss: 0.0020 - val_loss: 0.0021

Epoch 00005: val_loss did not improve from 0.00155
Epoch 6/20
1232/1232 [==============================] - 3359s 3s/step - loss: 0.0020 - val_loss: 0.00

In [ ]:
size = 512
w = 508
ww = int((w - size) / 2)

print(int((size - w) / 2))

In [ ]:
x_test_list = sorted(glob.glob(os.path.join(base_path, 'x_test', '*.npy')))
y_test_list = sorted(glob.glob(os.path.join(base_path, 'y_test', '*.npy')))

print(len(x_test_list), len(y_test_list))
print(x_test_list[0])

In [ ]:

test_idx = 28

x1_test = np.load(x_test_list[test_idx])
x1_test_resized = pyramid_expand(x1_test, 2)
y1_test = np.load(y_test_list[test_idx])
y_pred = model.predict(x1_test.reshape((1, 64, 64, 3)))


x1_test = (x1_test * 255).astype(np.uint8)
x1_test_resized = (x1_test_resized * 255).astype(np.uint8)
y1_test = (y1_test * 255).astype(np.uint8)
# y_pred = np.clip(y_pred.reshape((128, 128, 3)), 0, 1)
y_pred = y_pred.reshape((128, 128, 3))


x1_test = cv2.cvtColor(x1_test, cv2.COLOR_BGR2RGB)
x1_test_resized = cv2.cvtColor(x1_test_resized, cv2.COLOR_BGR2RGB)
y1_test = cv2.cvtColor(y1_test, cv2.COLOR_BGR2RGB)
y_pred = cv2.cvtColor(y_pred, cv2.COLOR_BGR2RGB)
y_pred = (y_pred * 255).astype('uint8')


plt.figure(figsize=(15, 10))
plt.subplot(1, 4, 1)
plt.title('input')
plt.imshow(x1_test)
plt.subplot(1, 4, 2)
plt.title('resized')
plt.imshow(x1_test_resized)
plt.subplot(1, 4, 3)
plt.title('output')
plt.imshow(y_pred)
plt.subplot(1, 4, 4)
plt.title('groundtruth')
plt.imshow(y1_test)